Updated: Oct. 26th 2022 by Fletcher Waller

# Mac Installation (Taken from CR Hayes notes:  Valid April 25, 2022):

## New GHOSTDR startup

Assumptions:
- you are running this locally. __For those working from Polaris (UVic's computing server) skip step 1.0 and 3.1__
- Already have anaconda installed
- Using python 3
- macOS Big Sur 11.6 (likely works with other setups, but just including for completeness)

#### NOTE
- depending on how you download it, the base directory might be called GHOSTDR or GHOSTDR-master
- Some of the locations of files and folders might be different depending on your machine
- if you try this on a windows machine, please let me know what steps you have to take and how they differed so I can add them here.

------------------

# 1.0 Setting up DRAGONS
If this doesn’t work for you check out this page for DRAGONS installation: https://www.gemini.edu/observing/phase-iii/understanding-and-processing-data/data-processing-software/download-latest#dragons

Once you have an anaconda version set up on your computer you should be able to do the following to create a conda environment called dragons (may differ on other operating systems)

    conda config --add channels http://ssb.stsci.edu/astroconda
    conda config --add channels http://astroconda.gemini.edu/public
    conda create -n dragons python=3.7 dragons stsci
    
### 1.1 running DRAGONS

Run `conda activate dragons` in your terminal to open a DRAGONS environment. __NOTE:  You have to do this every time that you are starting to work with dragons__

------------------

# 2.0 Setting up Gemini calibration manager


Go to your home directory and

    mkdir .geminidr
    cd .geminidr
    
Make a file called rsys.cfg, e.g.,
`touch rsys.cfg`. Open this file, and add the following:

    [calibs]
    standalone = True
    database_dir = <path to where you want the cal manager database>/cal_manager.db

then cd to that directory and while in your dragons environment do

    caldb init -v -w

### 2.1 Switching cal managers

It is recommended to have separate cal managers for different science object reductions, in order to keep calibration frames in order. To do this, you need to add a new line in `~/.geminidr/rsys.cfg`, and comment out any other pointings.

    [calibs]
    standalone = True
    #database_dir = <path to where you want the cal manager database>/cal_manager.db
    database_dir = <new path to where you want the cal manager database>/cal_manager.db
    
and then re-run `caldb init -v -w` just once in the desired directory. You should not need to re-initialize every time you switch cal managers, only the first time you make a new cal manager.

-------------------------

# 3.0 Installing the GHOST DR pipeline

Download from GHOSTDR github (https://github.com/ANU-RSAA/GHOSTDR) or Chris Hayes' version (https://github.com/hayescr/GHOSTDR)

Download options (Green CODE button):
- Download the Zip file and move the GHOSTDR-master directory to where you want it to be and unzip it.
- __OR__ Download with git via terminal “git clone https://github.com/<owner>/GHOSTDR.git”
    
Using vim or desired editor, add GHOSTDR to your python path in your ~/.zshrc or ~/.bashrc file (this file will be in your home directory):
    
    export PYTHONPATH=/<path to GHOSTDR>/GHOSTDR:$PYTHONPATH
    

    
### 3.1 Setup 

__NOTE:__ this step may not be necessary, (__and for those on polaris, is not possible__) as it simply sets up communication between DRAGONS and GHOSTDR. We are using a work-around at the moment by calling `--drpkg ghostdr` and `--adpkg ghost_instruments` in our commands later. this will ultimately not be necesary upon integration with DRAGONS.

do the following:

    cd <path>/GHOSTDR
    python setup.py install
    

Once you’ve done this and installed/set up DRAGONS you should be able to use GHOSTDR, you may also want to make a version of the manual (see below, or a compiled one that might be slightly out of data can be found here:  https://drive.google.com/file/d/1OqeAHZfAGdwETipcT-fGokUajllF_RnS/view?usp=sharing).  The manual will tell you the basics for how to use GHOSTDR, but I have provided a jupyter notebook walk-through below.


#### 3.2 Making the user manual

In GHOSTDR

    cd ghostdr/docs/usermanuals/GHOST_UsersManual
    make <option> (for example “make html”)
    
`Build finished. The HTML pages are in _build/html`

    cd _build/html/
    Open index.html


# 4.0 Directory Structure Organizing

There are a lot of files needed for and generated by GHOSTDR. For the sake of trouble shooting and starting over, I have developed a preferred directory structure for my GHOST data. I will show you a detailed walk-through of this structure, followed by a summary, and finally a notebook cell which will generate this file structure in full for you. You may use what you find useful. __NOTE:__ I highly recommend keeping your data __OUTSIDE__ of your GHOSTDR directory.

### Detailed walk-through

__Before you start copy-pasting, there is code below which will do all this automatically for you.__

Because the calibration manager works sometimes unpredictably, I find it is best to only include the calibrations you intend to use in your cal_manager. This can be achieved by placing only relevant calibration files in a directory with data, and by editing your rsys.cfg file to point to that directory.

At the top, I make the DATA directory.

    mkdir DATA/

Below, I make folders for individual objects (e.g. HD122563)

    mkdir DATA/HD122563/
    mkdir DATA/HR6000/

Within each object directory, if I have different readout modes (slow, norm) that I need to keep separate, I will make  directories for each of the modes. this is because the cal manager cannot differentiate between the two readmodes, and therefore will need different cal manager configurations in `rsys.cfg`.

    mkdir DATA/HD122563/norm/
    mkdir DATA/HD122563/slow/
    


`DATA/HD122563/norm/` is where we will be working with our data. At the end, this is where your reduced files will live.  An example of what my directory looks like is 

`DATA/HD122563/norm/:`

    calibrations/
    raw/
    intermediate/
    <reduced calibration .fits files, too numerous to mention here>
    cal_manager.db
    reduce.log
    
__These are made automatically by the reduction process:__ `calibrations/` is where the cal manager automatically stores your reduced calibration files. `cal_manager.db` is what your rysys.cfg file should point to. `reduce.log` is updated by GHOSTDR upon any reduction command.

__These are made manually:__ I explain `raw/` in the following section. `intermediate/` is where I dump some of the intermediate data products which can be useful, but I often do not want to look at. 
    

In order to keep reduced files separate from raw files, I make new directories here. 

    mkdir DATA/HD122563/norm/raw/
    mkdir DATA/HD122563/norm/raw/packed/
    mkdir DATA/HD122563/norm/raw/obj/

`DATA/HD122563/norm/raw/packed/` is the deepest the directories go. This is where you should download your files to. GHOST data is automatically bundled upon observation; an observation bundle is one fits file, which will contain an extension file for every exposure in the red, blue and slit cameras. This makes downloading and keeping files together easy. An example of what my packed directory looks like is

`DATA/HD122563/norm/raw/packed/:`

    bias_1x1.fits
    bias_2x4.fits
    flat_sr.fits
    arc_sr.fits
    HD122563_sr_2x4.fits
    

In order to reduce the data however, the bundle will need to be un-bundled. I find it is cleanest to unbundle into the directory one above, `DATA/HD122563/norm/raw/`. I will demonstrate this later. This way, if you need to start over for any reason, you can easily delete all reduced and unbundled files, while still leaving the bundled ones in place, eliminating the need to re-download them all again. 

The `DATA/HD122563/norm/raw/` directory is where every (_calibrations only_) blue, red, and slit camera exposure fits file will live (the slit files actually remain bundled themselves, because they contain sometiems hundreds of fits extensions). It is from here that you will create lists of files for GHOSTDR to reduce. The data here should be, for the most part, read-only. I don't recommend doing reductions and writing new files into this directory for any reason.

An example of what my `raw/` directory looks like is

`DATA/HD122563/norm/raw/:`

    packed/
    obj/
    bias_1x1_2x2_slit.fits
    bias_1x1_1x1_blue1.fits
    bias_1x1_1x1_blue2.fits
    bias_1x1_1x1_red1.fits
    bias_1x1_1x1_red2.fits
    bias_1x4_2x2_slit.fits
    bias_1x4_2x4_blue1.fits
    bias_1x4_2x4_blue2.fits
    bias_1x4_2x4_red1.fits
    bias_1x4_2x4_red2.fits
    flat_sr_2x2_slit.fits
    flat_sr_1x1_blue1.fits
    flat_sr_1x1_blue2.fits
    flat_sr_1x1_red1.fits
    flat_sr_1x1_red2.fits
    arc_sr_2x2_slit.fits
    arc_sr_1x1_blue1.fits
    arc_sr_1x1_blue2.fits
    arc_sr_1x1_red1.fits
    arc_sr_1x1_red2.fits
    reduce.log
    bias.1x1.blue
    bias.1x1.red
    bias.2x4.blue
    bias.2x4.red
    flat.sr.blue
    flat.sr.red
    arc.sr.blue
    arc.sr.red
    
The files which look like `arc.sr.red` are lists of relevant .fits files. We use these to simplify reduction, and I will show how to make them later.
    

Within `raw/`, I create a nested object directory `raw/obj/` which contains the un-bundled science exposures. I do this because there are no tags which distinguish object frames from calibration frames, so it is difficult to typewalk them into a list on their own. This may be confusing to you now, so just trust me on this one. An example of what my  `DATA/HD122563/norm/raw/obj/` looks like is 

`DATA/HD122563/norm/raw/obj/:`

    HD122563_sr_2x4_2x2_slit.fits
    HD122563_sr_2x4_blue1.fits
    HD122563_sr_2x4_blue2.fits
    HD122563_sr_2x4_red1.fits
    HD122563_sr_2x4_red2.fits
    sr.blue
    sr.red
    
### Summary

    /DATA/<star>/<readmode>/<working directory>/<unbundled calibrations>/<bundled files>
    /DATA/<star>/<readmode>/<working directory>/<unbundled calibrations>/<unbundled science>
    /DATA/<star>/<readmode>/<working directory>/cal_manager.db
    /This_Notebook.ipynb

## 4.1 Make your directory Structure

Name your main data directory, the objects you are reducing, and include the readmodes you need.   There are different readmodes because we took data using both the engineering GUI and the Sequence Executor, which had different defaults for the red camera. If you are unsure which you need, leave it as `['norm', 'slow']`

In [1]:
import GHOSTDR_tutorial_scripts as gts

In [ ]:
def tryMakePath(path):
    try:
        os.mkdir(path)
    except FileExistsError:
        print('Directory {} already exists'.format(path))
    else:
        print('Directory {} created'.format(path))

def makeDirectoryStructure(DATA_parent, obj_name, readmodes):
    """Create's Fletcher's favourite GHOSTDR directory structure for easy trouble-shooting. Does not add your data"""

    dirpath = os.path.join('./', DATA_parent,)
    tryMakePath(dirpath)
    for i in obj_name:
        objpath = os.path.join(dirpath,  i)
        tryMakePath(objpath)
        for j in readmodes:
            readmodepath = os.path.join(objpath,  j)
            tryMakePath(readmodepath)
            rawpath = os.path.join(readmodepath,  'raw')
            tryMakePath(rawpath)
            tryMakePath(os.path.join(readmodepath,  'intermediate'))
            tryMakePath(os.path.join(rawpath,  'packed'))
            tryMakePath(os.path.join(rawpath,  'obj'))    

In [4]:
###############################################################################################################
#User Input Cell
###############################################################################################################

DATAparent = 'DATA' # your main data directory, should be located OUTSIDE of GHOSTDR/
objnames = ['science1', 'science2'] # your science objects
readmodes = ['norm', 'slow'] # readmodes

In [5]:
gts.makeDirectoryStructure(DATAparent, objnames, readmodes)

Directory ./DATA1 created
Directory ./DATA1/science1 created
Directory ./DATA1/science1/norm created
Directory ./DATA1/science1/norm/raw created
Directory ./DATA1/science1/norm/intermediate created
Directory ./DATA1/science1/norm/raw/packed created
Directory ./DATA1/science1/norm/raw/obj created
Directory ./DATA1/science1/slow created
Directory ./DATA1/science1/slow/raw created
Directory ./DATA1/science1/slow/intermediate created
Directory ./DATA1/science1/slow/raw/packed created
Directory ./DATA1/science1/slow/raw/obj created
Directory ./DATA1/science2 created
Directory ./DATA1/science2/norm created
Directory ./DATA1/science2/norm/raw created
Directory ./DATA1/science2/norm/intermediate created
Directory ./DATA1/science2/norm/raw/packed created
Directory ./DATA1/science2/norm/raw/obj created
Directory ./DATA1/science2/slow created
Directory ./DATA1/science2/slow/raw created
Directory ./DATA1/science2/slow/intermediate created
Directory ./DATA1/science2/slow/raw/packed created
Director

# 5.0 Download Data 



### 5.1 Find the right data

I have organized all of these files in the following spreadsheet. Follow the instructions in the spreadsheet to get the correct data and calibration frame filenames. Then head over to the mirror (or regular GHOST directory if you are a Gemini person) and download the corresponding files.

https://docs.google.com/spreadsheets/d/1RlgwxeU2Hlj8gKo30QRWSIuzNlmnyYcHm2K-Wkdsc_Y/edit#gid=348066143

### 5.2 get access to the data

for non-Gemini people, the GHOST data is currently located on the GHOST mirror and can be accessed through ssh. To get access, send your public ssh key to Ed Chapin, Edward.Chapin@nrc-cnrc.gc.ca, and ask him to add it to the GHOST mirror. Then, enter the following into your terminal. 

    ssh ghostdr@206.12.91.101 
    
The data is located in the directory `/home/ghostdr/data_commissioning_mirror/observeDataPerm/`. It is a big list of every calibration and science frame taken during commissioning. 

### 5.3 Download the data 

Download your data into the `packed/` directory of the corresponding sicence object and readmode.